## Combining the CSV of all Participants

In [ ]:
#Basicsimport os
import sys
print (os.getcwd())
sys.path.append('/home/Hanna.Drimalla/ml-autism/scripts')

from __future__ import division

%matplotlib inline
import os
import pandas as pd
import datetime as dt
import imageio
import numpy as np
import subprocess
import seaborn as sns
import matplotlib.pyplot as plt
#from keras.models import Sequential, Model


#Stats
#import mne
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy
from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot

# Graphics
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.regression.mixed_linear_model import MixedLMParams
from statsmodels.stats.weightstats import DescrStatsW


#import pyvttbl as pt
import statsmodels.stats.api as sms
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
import warnings
from scipy.stats.mstats import zscore
#matplotlib.use('TkAgg')  
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as hac
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
#from sklearn.model_selection import cross_val_score
import mycharite
import mystats 

In [ ]:
study='charite'

In [ ]:
def smooth(df):
    round_df=df
    for i in set(df.vpn):
        for au in action_r:
            round_df.loc[round_df.vpn==i, au]=round_df.loc[round_df.vpn==i, au].rolling(30, win_type='triang').mean()
    return round_df

In [ ]:
# load dataframe
df, df_audio, action_r, action_c, gaze, audio =mycharite.load('charite')
df=mycharite.aq_charite(df)
df=mycharite.ados_charite(df)

print ('NT ' + str(sum(df.groupby('vpn').mean().asc==0)))
print ('ASC ' + str(sum(df.groupby('vpn').mean().asc==1)))

df=mycharite.exclude_outlier(df)

print ('NT ' + str(sum(df.groupby('vpn').mean().asc==0)))
print ('ASC ' + str(sum(df.groupby('vpn').mean().asc==1)))

df=mycharite.adapt_times(df)

In [ ]:
var=['gaze_angle_x', 'gaze_angle_y']
df_speed_acc=mycharite.calc_speed(df, var)
df=mycharite.calc_gaze_var(df)

In [ ]:
df=pd.merge(df, df_speed_acc, on=['vpn', 'conversation'], how='left')

In [ ]:
new_features=['gaze_angle_x', 'gaze_angle_y',
              'gaze_angle_x_centered', 'gaze_angle_y_centered',
              'gaze_angle_x_abs', 'gaze_angle_y_abs', 
              'gaze_angle_total_centered_abs',
              'mean_speed_gaze_angle_x', 'mean_acc_gaze_angle_x',
              'var_speed_gaze_angle_x', 'var_acc_gaze_angle_x', 
              'mean_speed_gaze_angle_y', 'mean_acc_gaze_angle_y', 
              'var_speed_gaze_angle_y', 'var_acc_gaze_angle_y']

## Descriptives

In [ ]:
for i in new_features:
    print (i)
    print (df.groupby('vpn').mean().groupby('asc')[i].describe())

In [ ]:
def group_comparison(df):
    var=['mean_speed_gaze_angle_x', 'mean_acc_gaze_angle_x',
              'mean_speed_gaze_angle_y', 'mean_acc_gaze_angle_y']
    
    

    for i in var:
        print (i)
        df1=df[df.asc==0].groupby('vpn').mean()
        df2=df[df.asc==1].groupby('vpn').mean()
        mystats.two_ind_sample_tests(df1, df2, i)
        
group_comparison(df)

# Smooth

In [ ]:
def smooth(df):
    df.index = pd.to_datetime(df['timepoint'].astype('float64'), unit='s')
    df=df.groupby(['vpn']).resample('S').mean().reset_index(drop=True)
    return df

In [ ]:
df_small=df.groupby(['vpn', 'conversation']).mean().reset_index()
df_small=pd.merge(df_small, df_speed_acc, on=['vpn', 'conversation'])

# Graphs

In [ ]:
def time_plot(df, var, title):
    plt.plot(np.array(df.reset_index(drop=True).groupby(['counter']).mean()[var]), color='green')
    plt.plot(np.array(df[df.asc==0].reset_index(drop=True).groupby(['counter']).mean()[var]), color='blue')
    #scale!!!
    plt.title(str(title))
    plt.show()

In [ ]:
for con in np.unique(df.conversation):
    time_plot(df[df.conversation==con], 'gaze_angle_x', con)

In [ ]:
for con in np.unique(df.conversation):
    time_plot(df[df.conversation==con], 'gaze_angle_y', con)

In [ ]:
def gaze_heat(df, title):
    plt.close()
    sns.kdeplot(df.gaze_angle_x_centered, df.gaze_angle_y_centered, cmap="Reds", shade=True, shade_lowest=False)
    plt.ylim(-0.1, 0.1)
    plt.xlim(-0.2, 0.2)
    plt.yticks(fontsize=16)    
    plt.xticks(fontsize=16)  
    plt.xlabel('X-Angle of Gaze', fontsize=18) 
    plt.ylabel(('Y-Angle of Gaze'), fontsize=18)
    plt.legend(loc='upper left', fontsize=16)  
    plt.savefig('Gaze_'+ str(title) + '_.png')
    plt.show() 


In [ ]:
def gaze_heat_not_centered(df, title):
    plt.close()
    sns.kdeplot(df.gaze_angle_x, df.gaze_angle_y, cmap="Reds", shade=True, shade_lowest=False)
    plt.ylim(-0.1, 0.1)
    plt.xlim(-0.2, 0.2)
    plt.yticks(fontsize=16)    
    plt.xticks(fontsize=16)  
    plt.xlabel('X-Angle of Gaze', fontsize=18) 
    plt.ylabel(('Y-Angle of Gaze'), fontsize=18)
    plt.legend(loc='upper left', fontsize=16)  
    plt.savefig('Gaze_'+ str(title) + '_.png')
    plt.show() 


In [ ]:
gaze_heat(df[df.asc==1].reset_index(drop=True), 'ASC')
gaze_heat(df[df.asc==0].reset_index(drop=True), 'ASC')

In [ ]:
gaze_heat_not_centered(df[df.asc==1].reset_index(drop=True), 'ASC')
gaze_heat_not_centered(df[df.asc==0].reset_index(drop=True), 'ASC')

In [ ]:
plt.hist(df[df.asc==0].groupby('vpn').max().gaze_angle_y)
plt.show()

plt.hist(df[df.asc==1].groupby('vpn').max().gaze_angle_y)
plt.show()

In [ ]:
plt.hist(df[df.sex==1][df.asc==0].groupby('vpn').mean().gaze_angle_y)
plt.show()

plt.hist(df[df.sex==1][df.asc==1].groupby('vpn').mean().gaze_angle_y)
plt.show()

In [ ]:
#df.groupby('vpn').max().counter

In [ ]:
window=1000
i=10
plt.plot(df[df.vpn==i]['gaze_angle_y'].reset_index(drop=True).rolling(window=window).mean().reset_index(drop=True), color='blue')
#plt.plot(df[df.vpn==i]['gaze_angle_x'], color='red')

#plt.ylim(-0.15, 0.15)
#plt.plot(np.mean(speeds_nt.iloc[:, 0:-2].rolling(window=window).mean(), axis=0), color='green')
plt.show()

In [ ]:
accs_asc=accs[(accs.asc==1)]
accs_nt=accs[(accs.asc==0)]

plt.plot(np.mean(accs_asc.iloc[:,0:-2], axis=0), color='blue')
plt.plot(np.mean(accs_nt.iloc[:, 0:-2], axis=0), color='green')
plt.show()

In [ ]:
plt.hist(((df[df.asc==0].groupby('vpn').mean().gaze_angle_x_centered_abs)))
plt.show()

In [ ]:
plt.hist(((df[df.asc==1].groupby('vpn').mean().gaze_angle_x_centered_abs)))
plt.show()

In [ ]:
plt.scatter(df.groupby('vpn').mean()['gaze_angle_x_centered_abs'],
           df.groupby('vpn').mean()['asq'])
plt.show()

# Calculations 

### Group Comparsiond: Gaze Variation

In [ ]:
# 1 is female

var = ['gaze_angle_y',
       'gaze_angle_x', 'gaze_angle_x_abs',
       'gaze_angle_y_abs', 
       'mean_speed_gaze_angle_x', 'mean_acc_gaze_angle_x',
       'mean_speed_gaze_angle_y', 'mean_acc_gaze_angle_y']

df_small=df_small[df_small.conversation!='intro']
df_small=df_small[df_small.vpn!=1]
   
for v in var:
    print (v)
    print (mystats.two_ind_sample_tests(df_small[(df_small.asc==0)].groupby('vpn').mean(), 
                         df_small[(df_small.asc==1)].groupby('vpn').mean(), v))

In [ ]:
def correlations (df, traits, gaze_features):
    for trait in traits:

        for gaz in gaze_features:
            try:
                print (gaz)
                print (trait)
                a=df[df[trait]>0].groupby('vpn').mean()[[trait, gaz]].dropna()
                mystats.correlation(a[trait], a[gaz])
                plt.scatter(a[trait], a[gaz])
                plt.show()
            except: 
                print ('not calculable')
       
        
gaze_features=['gaze_angle_y',
       'gaze_angle_x', 'gaze_angle_x_abs',
       'gaze_angle_y_abs',
       'mean_speed_gaze_angle_x', 'mean_acc_gaze_angle_x',
       'mean_speed_gaze_angle_y', 'mean_acc_gaze_angle_y']

traits=['ados_commu', 'ados_social', 'ados_total', 'adir_social', 'adir_commu', 'adir_behav',
       'adir_onset']

correlations (df_small, traits, gaze_features)

In [ ]:
def graphics(df):
    df_nt=df_small[df_small.asc==0].groupby(['vpn']).mean().reset_index(drop=True)
    df_asc=df_small[df_small.asc==1].groupby(['vpn']).mean().reset_index(drop=True)

    var = ['gaze_angle_x',
           'gaze_angle_y', 'gaze_angle_x_abs',
           'gaze_angle_y_abs', 
           'mean_speed_gaze_angle_x', 'mean_acc_gaze_angle_x',
           'var_speed_gaze_angle_x', 'var_acc_gaze_angle_x',
           'mean_speed_gaze_angle_y', 'mean_acc_gaze_angle_y',
           'var_speed_gaze_angle_y', 'var_acc_gaze_angle_y']

    for v in var:
        print (v)
        #mystats.two_ind_sample_tests(df_nt, df_asc, v)
        plt.hist(df_nt[v], color='blue')
        plt.hist(df_asc[v], color='red')
        plt.show()

In [ ]:
# ACHTUNG: 1 is female!!!

def gender_analysis(df):
    gender=[1, 2]
    var = ['gaze_angle_y',
           'gaze_angle_x', 'gaze_angle_x_abs',
           'gaze_angle_y_abs', 
           'mean_speed_gaze_angle_x', 'mean_acc_gaze_angle_x',
           'mean_speed_gaze_angle_y', 'mean_acc_gaze_angle_y']
    df_small=df_small[df_small.conversation!='intro']

    for g in gender:
        print (g)
        print (len(df_small[ (df_small.sex==g) &(df_small.asc==0)].groupby('vpn').mean()))
        print (len(df_small[ (df_small.sex==g) &(df_small.asc==1)].groupby('vpn').mean()))
        for v in var:
            print (v)
            print (mystats.two_ind_sample_tests(df_small[ (df_small.sex==g) &(df_small.asc==0)].groupby('vpn').mean(), 
                                 df_small[(df_small.sex==g) &(df_small.asc==1)].groupby('vpn').mean(), v))


    gender=[1, 2]
    var = ['gaze_angle_y',
           'gaze_angle_x', 'gaze_angle_x_abs',
           'gaze_angle_y_abs', 
           'mean_speed_gaze_angle_x', 'mean_acc_gaze_angle_x',
           'mean_speed_gaze_angle_y', 'mean_acc_gaze_angle_y']
    df_small=df_small[df_small.conversation!='intro']
    for g in gender:
        print (g)
        for v in var:
            print (v)
            print (mystats.two_ind_sample_tests(df_small[ (df_small.sex==g) &(df_small.asc==0)].groupby('vpn').mean(), 
                                 df_small[(df_small.sex==g) &(df_small.asc==1)].groupby('vpn').mean(), v))

    print (np.mean(df[(df.sex==1) & (df.asc==0)].groupby('vpn').mean().gaze_angle_y))
    print (np.mean(df[(df.sex==1) &(df.asc==1)].groupby('vpn').mean().gaze_angle_y))
    print (np.mean(df[(df.sex==2) & (df.asc==0)].groupby('vpn').mean().gaze_angle_y))
    print (np.mean(df[(df.sex==2) &(df.asc==1)].groupby('vpn').mean().gaze_angle_y))
    print (np.mean(df[(df.sex==1) & (df.asc==0)].groupby('vpn').std().gaze_angle_y))
    print (np.mean(df[(df.sex==1) &(df.asc==1)].groupby('vpn').std().gaze_angle_y))
    print (np.mean(df[(df.sex==2) & (df.asc==0)].groupby('vpn').std().gaze_angle_y))
    print (np.mean(df[(df.sex==2) &(df.asc==1)].groupby('vpn').std().gaze_angle_y))


In [ ]:
df.columns
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
df['conversationpart'] = LE.fit_transform(df['conversation'])

### seperated by Speaker

In [ ]:
for i in set(df.whospeaks):

    df_nt=df[df.whospeaks==i][df.asc==0].groupby(['vpn']).mean().reset_index(drop=True)
    df_asc=df[df.whospeaks==i][df.asc==1].groupby(['vpn']).mean().reset_index(drop=True)


    mystats.two_ind_sample_tests(df_nt, df_asc, 'gaze_angle_x_centered')
    mystats.two_ind_sample_tests(df_nt, df_asc, 'gaze_angle_y_centered')


In [ ]:
speeds_asc=speeds[(speeds.asc==1)]
speeds_nt=speeds[(speeds.asc==0)]

print scipy.stats.ttest_ind(np.mean(speeds_asc.iloc[:, 0:-2], axis=1),
                           np.mean(speeds_nt.iloc[:, 0:-2], axis=1))#, color='green')
print np.mean(np.mean(speeds_asc.iloc[:, 0:-2]))
print np.mean(np.mean(speeds_nt.iloc[:, 0:-2]))
        
print scipy.stats.ttest_ind(np.var(speeds_asc.iloc[:, 0:-2], axis=1),
                           np.var(speeds_nt.iloc[:, 0:-2], axis=1))#, color='green')

### over whole conversation

In [ ]:
data=df[df.whospeaks=='proband'].groupby(['vpn', 'conversationpart']).mean().reset_index()
model = sm.MixedLM.from_formula('gaze_angle_x_centered' + " ~ asc", data, groups=data['vpn']) #, re_formula="~conversationpart")
mdf = model.fit()
print(mdf.summary())